In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm, trange
import yaml
import os
import seaborn as sns
sns.set_theme()

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from model import GRU_CNN
from dataset import load_NASA

# Load the YAML configuration file
with open('config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

# # Access the variables
NUM_CYCLES = cfg['NUM_CYCLES']
NUM_PREDS = cfg['NUM_PREDS']
FEATURE_DIM1 = cfg['FEATURE_DIM1']
FEATURE_DIM2 = cfg['FEATURE_DIM2']
NUM_ATTENTION = cfg['NUM_ATTENTION']
EPOCHS = cfg['EPOCHS']
LEARNING_RATE = cfg['LEARNING_RATE']
BATCH_SIZE = cfg['BATCH_SIZE']

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load data
train_dataset, test_dataset = load_NASA(folder='NASA_DATA', num_cycles=1, split_ratio=0.5, scale_data=True)

# Train/test split
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [2]:
# NN model
model = GRU_CNN().to(device)

# Loss function and optimizer
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# Training loop
best_epoch = 0
best_loss = float('inf')
Loss_log = []
os.makedirs('models', exist_ok=True)
model.train()
t_range = trange(EPOCHS)
for epoch in t_range:
    train_losses = []
    for inputs, outputs in train_dataloader:
        inputs = inputs.float().to(device).squeeze(1).permute(0,2,1)
        outputs = outputs.float().to(device)
        predicted_outputs = model(inputs)
        optimizer.zero_grad()
        loss = criterion(predicted_outputs, outputs)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    test_losses = []
    for inputs, outputs in test_dataloader:
        inputs = inputs.float().to(device).squeeze(1).permute(0,2,1)
        outputs = outputs.float().to(device)
        with torch.no_grad():
            predicted_outputs = model(inputs)
            test_loss = criterion(predicted_outputs, outputs)
            test_losses.append(test_loss.item())
    Loss_log.append([np.mean(train_losses),np.mean(test_losses)])
    # Print the loss for monitoring after each epoch
    t_range.set_description(f"train loss: {np.mean(train_losses)}, test loss: {np.mean(test_losses)}")
    t_range.refresh()
    # Check if the current loss is the best so far
    if np.mean(test_losses) < best_loss:
        best_epoch = epoch
        best_loss = np.mean(test_losses)
        torch.save(model, f'models/trained_model_{best_loss:.6f}_{best_epoch}.pt')
Loss_log = np.array(Loss_log)
np.save(f'models/training_log_{NUM_CYCLES}_{NUM_PREDS}.npy', Loss_log)
plt.figure(figsize=(8,5))
plt.plot(Loss_log[0,:best_epoch])
plt.plot(Loss_log[1,:best_epoch])
plt.legend(["Train Loss","Test Loss"])
plt.grid("on")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.show()

  0%|          | 0/2000 [00:00<?, ?it/s]c:\Users\HeydarianArdakaniA\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\conv.py:304: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:1032.)
  return F.conv1d(input, weight, bias, self.stride,
train loss: 0.005191770192511656, test loss: 0.05389964833323445:  50%|█████     | 1006/2000 [17:15<17:02,  1.03s/it]  


KeyboardInterrupt: 

In [22]:
model = torch.load(f'models/trained_model_0.037717_595.pt')
# Train/test split
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

error_data_MAE = []
error_data_MSE = []
error_data_RMSE = []

criterion1 = nn.L1Loss()
criterion2 = nn.MSELoss()

for inputs, outputs in test_dataloader:
    inputs = inputs.float().to(device).squeeze(1).permute(0,2,1)
    outputs = outputs.float().to(device)
    with torch.no_grad():
        predicted_outputs = model(inputs)
        MAE_L = criterion1(predicted_outputs, outputs)
        MSE_L = criterion2(predicted_outputs, outputs)
        error_data_MAE.append(MAE_L.item())
        error_data_MSE.append(MSE_L.item())
        error_data_RMSE.append(np.sqrt(MSE_L.item()))

error_data_MAE = np.array(error_data_MAE)
error_data_MSE = np.array(error_data_MSE)
error_data_RMSE = np.array(error_data_RMSE)


C:\Users\HeydarianArdakaniA\AppData\Local\Temp\ipykernel_7284\270819026.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f'models/trained_model_0.03771

In [23]:
print(f"Average - MSE:{np.mean(error_data_MSE):.6f}, RMSE:{np.mean(error_data_RMSE):.6f}, MAE:{np.mean(error_data_MAE):.6f}, MAX_ERR:{np.max(error_data_MAE):.6f}")


Average - MSE:0.004138, RMSE:0.060561, MAE:0.037229, MAX_ERR:0.059465


In [13]:
import pandas as pd

charge_data = pd.read_csv('NASA_DATA'+'/charge_data.csv')

In [18]:
np.unique(charge_data['Battery_id'])

array(['B0005', 'B0006', 'B0007', 'B0018', 'B0025', 'B0027', 'B0028',
       'B0029', 'B0030', 'B0031', 'B0032', 'B0034', 'B0036', 'B0045',
       'B0049', 'B0051', 'B0053', 'B0054', 'B0055', 'B0056'], dtype=object)